In [6]:
# !pip install google-search-results

# src/tools.py
# from pypatent import Patent # Commented out

import json
import os
from serpapi import GoogleSearch # New import for Google Search

# Commented out the original patent_search function
# def patent_search(query: str, max_results=3) -> list:
#     """
#     Searches for patents using the pypatent library and returns structured results.
#     This is a tool that the agent can decide to call.
#     """
#     print(f"--- TOOL: Executing Patent Search with query: '{query}' ---")
#     found_patents = []
#     try:
#         search_results = Patent(query)
#         search_results.get_patents(max_page=1) # Fetch one page of results
#
#         if not search_results.patents:
#             return "No patents found for this query."
#
#         for patent in search_results.patents[:max_results]:
#             patent_details = {
#                 "title": patent.get('title'),
#                 "patent_number": patent.get('patent_number'),
#                 "abstract": patent.get('abstract'),
#                 "url": patent.get('link'),
#                 "claims": patent.get('claims'),
#                 "description": patent.get('description')
#             }
#             found_patents.append(patent_details)
#         return found_patents
#     except Exception as e:
#         return f"An error occurred during patent search: {e}"

def search(query: str, num_results=5) -> list:
    """
    Performs a Google Patents search using SerpApi and returns a list of structured patent results.
    This is a tool that the agent can decide to call.
    """
    print(f"--- TOOL: Executing Google Patents Search with query: '{query}' ---")
    try:
        params = {
            "engine": "google_patents", # CHANGED: from "google" to "google_patents"
            "q": query,
            "api_key": os.environ.get("SERPAPI_API_KEY"),
            "num": num_results
        }
        search_engine = GoogleSearch(params) # Renamed variable to avoid conflict with function name
        results = search_engine.get_dict()

        formatted_results = []
        if "organic_results" in results: # Google Patents results are still often under 'organic_results' or similar
            for result in results["organic_results"]:
                patent_details = {
                    "title": result.get("title"),
                    "link": result.get("link"),
                    "abstract": result.get("snippet"), # Snippet often serves as the abstract
                    "patent_number": result.get("patent_number"),
                    "publication_date": result.get("publication_date"),
                    "assignee": result.get("assignee"),
                    # You can add more fields if needed, e.g., inventors, filing_date
                    # "inventors": result.get("inventors"),
                    # "filing_date": result.get("filing_date"),
                }
                # Filter out None values for cleaner output
                patent_details = {k: v for k, v in patent_details.items() if v is not None}
                formatted_results.append(patent_details)
        else:
            return "No patent search results found."

        return formatted_results
    except Exception as e:
        return f"An error occurred during Google Patents search: {e}"


def final_report(model, invention_text: str, search_results: list) -> str:
    """
    This tool takes the original invention and the search results,
    then calls the generative AI model with a specific prompt to generate
    the final, structured analysis report.

    Args:
        model: The generative AI model instance (e.g., Gemini).
        invention_text: The full text of the user's invention disclosure.
        search_results: The list of prior art found by the search tool (now Google Patents).

    Returns:
        A string containing the formatted final report.
    """
    print("--- TOOL: Executing Final Report Generation ---")

    # Format the search results into a readable string for the prompt
    formatted_results = json.dumps(search_results, indent=2)

    # 1. Construct the final, detailed prompt
    final_prompt = FINAL_REPORT_PROMPT.format(
        invention_text=invention_text,
        search_results=formatted_results
    )

    # 2. Call the generative AI model to generate the report
    print("    - Calling Gemini to write the final analysis...")
    report = model.generate_content(final_prompt).text

    # 3. Return the generated report
    return report
# src/prompts.py

# This prompt guides the AI to break down the user's request into a series of steps (a plan).
# It uses the ReAct (Reason+Act) format.
REACT_PLANNING_PROMPT = """
You are an AI agent that assists with patent prior art searches. Your goal is to analyze a document describing an invention and see if you can find documents that describe the features of the invention that existed
prior to the date of the invention document.  Documents that are on or after the date of the invention document do not matter.

You must break down your task into a series of thoughts and actions.

**Crucial Instruction:** Your *final line* for each step should be either a "Thought:" or an "Action:".
If you decide to take an action, it *must* be formatted exactly as `Action: tool_name(tool_input)`.

**Available Tools:**
- **`search`**: Use this tool to perform a **Google Patents search**. The input to this tool should be a concise search query string based on the invention's key technical concepts. Example: `Action: search("electric vehicle charging system prior art")` # UPDATED: Clarified "Google Patents search" and example
- **`final_report`**: Use this tool ONLY when you have gathered enough information from your patent search. It takes the original invention text and the search results to generate the final analysis. Example: `Action: final_report()`

Feel free to use search Engines and any other tools that make sense.  Only rely upon documents that were created before the date of the invention disclosure.


**Reasoning Process:**
1.  **Thought:** Start by analyzing the user's request. My first step is to understand the core technology of the invention disclosure.
2.  **Action:** Based on my understanding, I will formulate a search query and use the `search` tool to find **prior art patents**. Put emphasis on the features of claim 1 of the invention disclosure and formulate the query specifically for patent databases.
3.  **Observation:** I will analyze the **patent search results** from the `search` tool.
4.  **Thought:** I will decide if I have enough information. If the results are highly relevant, I may have enough. If not, I might try a different patent search query.
5.  **Action:** Once I have sufficient information, I will use the `final_report` tool to generate the complete analysis.

Here is the user's request. Begin your thought process.

**Invention Disclosure:**
{user_input}
"""

# This prompt is used for the final step to generate a comprehensive report.
FINAL_REPORT_PROMPT = """
You are a patent analyst AI. Your task is to provide a patentability
assessment based on an invention disclosure and a list of **prior art patent documents** you have found.

**INVENTION DISCLOSURE:**
{invention_text}

**PRIOR ART PATENT SEARCH RESULTS:** # UPDATED HEADING
{search_results}

**Instructions:**
1.  **Overall Assessment:** Start with the features of claim 1 of the invention disclosure.
2.  **Analysis of Novelty:** For each piece of prior art patent document found, explain how the document's text (including title, abstract, patent number, and assignee) might challenge the features of the invention described in claim 1.
3.  **Key Distinguishing Features:** Identify which features of claim 1 of the invention disclosure appear to be novel and not explicitly mentioned in the prior art patent documents.
4.  **Recommendation:** Conclude with a recommendation on whether the prior art patent references appear to disclose the claimed features.

Analyze the full available details of the prior art references (title, abstract, patent number, publication date, assignee) to determine if the features of claim 1 of the invention disclosure are found within the references.

Structure your response using clear headings and bullet points.
"""


# src/memory.py

class AgentMemory:
    """A simple class to store the agent's conversation history."""
    def __init__(self):
        self._history = []

    def add_entry(self, entry: str):
        """Adds a new thought, action, or observation to the memory."""
        self._history.append(entry)

    def get_history(self) -> str:
        """Returns the entire history as a formatted string."""
        return "\n".join(self._history)

# !pip install google.api_core

# src/agent.py
import json
import re
import time
import logging
import os

#from tools import search, final_report # Tool import remains 'search'
#from prompts import REACT_PLANNING_PROMPT, FINAL_REPORT_PROMPT
#from memory import AgentMemory


import tenacity
from google.api_core import exceptions as google_exceptions

class GeminiPatentAgent:
    """The core agent for handling patent analysis."""

    def __init__(self, model):
        """
        Initializes the agent.
        Args:
            model: An instance of a generative AI model (like Google's Gemini).
        """
        self.model = model
        self.memory = AgentMemory()
        self.available_tools = {
            "search": search, # Tool mapping remains 'search'
            "final_report": final_report
        }
        print("--- Gemini Patent Agent Initialized ---")

    @tenacity.retry(
        retry=tenacity.retry_if_exception_type(google_exceptions.ResourceExhausted),
        wait=tenacity.wait_exponential(multiplier=1, min=2, max=60),
        stop=tenacity.stop_after_attempt(10),
        before_sleep=tenacity.before_sleep_log(logging.getLogger(__name__), logging.INFO)
    )
    def _call_gemini(self, prompt):
        """
        A wrapper for calling the generative model.
        This method is now decorated with retry logic.
        """
        print("    - Calling Gemini API...")
        response = self.model.generate_content(prompt)
        print("    - ...API call successful.")
        return response.text


    def _parse_action(self, llm_output: str):
        """Uses regex to parse the LLM's output for an action."""
        match = re.search(r"Action:\s*(\w+)\s*\((.*)\)\s*$", llm_output, re.DOTALL | re.MULTILINE)
        if match:
            tool_name = match.group(1).strip()
            tool_input_str = match.group(2).strip()
            tool_input = tool_input_str.strip("'\"") if tool_input_str else None
            print(" tool_name from _parse_action is ", tool_name)
            return tool_name, tool_input
        else:
            print(" This is not a match in _parse_action and tool name is not found from LLM response!")
            print(f" LLM Output that failed to parse:\n{llm_output}")
            return None, None

    def run(self, user_input: str, max_iterations=5):
        """
        Runs the agent's core ReAct (Reason + Act) loop.
        """
        print(f"\n--- Running Agent with Input ---")

        planning_prompt = REACT_PLANNING_PROMPT.format(user_input=user_input)
        self.memory.add_entry(f"User Input: {user_input}")
        observation_results = {}

        for i in range(max_iterations):
            print(f"\n--- Iteration {i+1} ---")

            current_prompt = f"{planning_prompt}\n{self.memory.get_history()}"

            try:
                llm_response = self._call_gemini(current_prompt)
            except google_exceptions.ResourceExhausted as e:
                print("--- AGENT ERROR: API rate limit exceeded after multiple retries. Aborting. ---")
                print(f"--- Last error: {e} ---")
                return "Agent failed due to persistent API rate limiting."

            self.memory.add_entry(f"LLM Response:\n{llm_response}")
            print(llm_response)

            tool_name, tool_input = self._parse_action(llm_response)

            if tool_name in self.available_tools:
                if tool_name == "final_report" and not observation_results.get("search"): # Condition remains 'search'
                    no_results_entry = "Observation: The `final_report` tool cannot be called yet because no prior art has been found. You must use the `search` tool first, or try a different search query."
                    self.memory.add_entry(no_results_entry)
                    print(f"\n[GUARDRAIL ACTIVATED] {no_results_entry}")
                    continue

                tool_function = self.available_tools[tool_name]

                if tool_name == "search": # Tool name remains 'search'
                    result = tool_function(tool_input)
                    observation_results[tool_name] = result
                    observation_entry = f"Observation: Tool `{tool_name}` returned: {json.dumps(result, indent=2)}"
                    self.memory.add_entry(observation_entry)
                    print(observation_entry)

                elif tool_name == "final_report":
                    final_analysis = tool_function(
                        model=self.model,
                        invention_text=user_input,
                        search_results=observation_results.get("search", []) # Key remains 'search'
                    )
                    return final_analysis
            else:
                print("--- Agent did not choose a valid action. Ending loop. ---")
                return "The agent could not complete the request because it failed to choose a valid action."

        return "Agent reached maximum iterations without finishing."


# !pip install google.generativeai

# main.py
import google.generativeai as genai
import os
#from agent.GeminiPatentAgent import GeminiPatentAgent

# --- Configuration ---
# Get your API key from environment variables or secure storage
#os.environ['GOOGLE_API_KEY'] =
try:
    API_KEY = "PRIVATE"##os.environ['GOOGLE_API_KEY']
    # Add SerpApi key
    # It's highly recommended to set this as an environment variable
    # e.g., export SERPAPI_API_KEY="YOUR_SERPAPI_KEY"
    SERPAPI_API_KEY = "PRIVATE"
    if not SERPAPI_API_KEY:
        raise KeyError("SERPAPI_API_KEY environment variable not set.")

except KeyError as e:
    print(f"ERROR: {e}")
    exit()

genai.configure(api_key=API_KEY)
# Using a text-focused model suitable for this task
model = genai.GenerativeModel('gemini-2.5-pro')

def read_invention_disclosure(file_path: str):
    """Reads the invention disclosure from a file."""
    try:
        with open(file_path, 'r') as f:
            return f.read()
    except FileNotFoundError:
        print(f"Error: Invention disclosure file not found at '{file_path}'")
        return None

if __name__ == "__main__":
    # 1. Receive User Input (by reading the file)
    invention_file = "invention_disclosure.txt"
    invention_details = read_invention_disclosure(invention_file)

    if invention_details:
        # 2. Initialize and Run the Agent
        patent_agent = GeminiPatentAgent(model=model)
        final_response = patent_agent.run(user_input=invention_details)

        print("======================================================")
        print(final_response)
        # 5. Print the Final Response
        print("\n======================================================")
        print("===        FINAL PATENTABILITY ANALYSIS        ===")
        print("======================================================")
        print(final_response)

--- Gemini Patent Agent Initialized ---

--- Running Agent with Input ---

--- Iteration 1 ---
    - Calling Gemini API...
    - ...API call successful.
Thought:
I need to find prior art for U.S. Patent No. 5,970,479 A, which has a priority date of October 28, 1994. My search must be limited to documents published before this date.

The core of the invention, as described in claim 1, is a computer-based system for formulating customized multi-party risk management contracts. The key inventive step appears to be the mechanism for pricing and matching. It involves two different types of stakeholders providing independent inputs:
1.  An "ordering stakeholder" (a buyer) defines the terms of a contract, including the underlying event (phenomenon), the possible outcomes, and the specific "entitlements" (payouts) for each outcome. They also offer a "consideration" (premium) they are willing to pay.
2.  A "counter-party stakeholder" (a seller) independently provides "registering data," which r